In [1]:
!git clone https://github.com/likr/kplib

Cloning into 'kplib'...
remote: Enumerating objects: 20334, done.
remote: Total 20334 (delta 0), reused 0 (delta 0), pack-reused 20334
Receiving objects: 100% (20334/20334), 142.24 MiB | 31.23 MiB/s, done.
Resolving deltas: 100% (13/13), done.
Updating files: 100% (20801/20801), done.


In [2]:
!pip install ortools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.8/24.8 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 8.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-core 1.34.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<4.0.0dev,>=3.19.5, but you have protobuf 5.26.1 which is incompatible.
googleapis-common-protos 1.63.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.1,!

In [3]:
from ortools.algorithms.python import knapsack_solver
import os, random, time


In [4]:
solver = knapsack_solver.KnapsackSolver(
    knapsack_solver.SolverType.KNAPSACK_MULTIDIMENSION_BRANCH_AND_BOUND_SOLVER,
    "KnapsackExample",
)

In [5]:
def rmdir(path):
    if (not os.path.isdir(path)): return

    fl = os.listdir(path)
    for i in fl:
        subPath = path + "/" + i
        if (os.path.isdir(subPath)):
            rmdir(subPath)
            continue
        os.remove(subPath)

    os.rmdir(path)


srcPath = "/content/kplib" # test cases folder
destPath = "/content/picked_test_cases" # picked test cases folder

RGroup = "R10000"

rmdir(destPath) # remove old test_cases folder
os.mkdir(destPath)

for i in os.listdir(srcPath):
    if(i == ".git" or i == "README.md"):
        continue
    os.mkdir(destPath + "/" + i)
    for j in os.listdir(srcPath + "/" + i):
        srcTestCase = os.listdir(srcPath + "/" + i + "/" + j + "/" + RGroup)
        os.rename(srcPath + "/" + i + "/" + j + "/" + RGroup + "/" + srcTestCase[random.randint(0, len(srcTestCase) - 1)], destPath + "/" + i + "/" + j + ".kp")

In [6]:
def solve(oSrcFile, oResFile, timeLimit):
    values = []
    weights = [[]]
    capacities = []

    packed_items = []
    packed_weights = []
    total_weight = 0

    hData = oSrcFile.readlines()

    capacities.append(int(hData[2]))

    for i in range(4, 4 + int(hData[1])):
        values.append(int(hData[i].split(" ")[0]))
        weights[0].append(int(hData[i].split(" ")[1]))

    st = time.time()

    solver.init(values, weights, capacities)
    solver.set_time_limit(timeLimit)

    computed_value = solver.solve()

    elapsedTime = time.time() - st

    for i in range(len(values)):
        if solver.best_solution_contains(i):
            packed_items.append(i)
            packed_weights.append(weights[0][i])
            total_weight += weights[0][i]

    oResFile.write("Size: " + str(len(values)) + " items, Time taken: " + str(elapsedTime) + "s")
    if (elapsedTime > timeLimit):
        oResFile.write(" (time limit exceeded)")
    oResFile.write("\n")
    oResFile.write("Total value = " + str(computed_value) + "\n")
    oResFile.write("Total weight = " + str(total_weight) + "\n")
    oResFile.write("Total packed items = " + str(len(packed_items)) + "\n")
    oResFile.write("\n\n")

    oSrcFile.close()


def main():
    if not os.path.exists("/content/_OR_Tools_Results/"):
        os.makedirs("/content/_OR_Tools_Results/")
    lst = [
        "00Uncorrelated",
        "01WeaklyCorrelated",
        "02StronglyCorrelated",
        "03InverseStronglyCorrelated",
        "04AlmostStronglyCorrelated",
        "05SubsetSum",
        "06UncorrelatedWithSimilarWeights",
        "07SpannerUncorrelated",
        "08SpannerWeaklyCorrelated",
        "09SpannerStronglyCorrelated",
        "10MultipleStronglyCorrelated",
        "11ProfitCeiling",
        "12Circle"

    ]
    for test in range(0,13):
        testGroupName = lst[test]  # Chọn 1 trong 13 group trong list trên để solve
        testGroupPath = "/content/picked_test_cases/" + testGroupName
        testCaseFile = os.listdir(testGroupPath)

        resultFile = open("/content/_OR_Tools_Results/" + testGroupName + ".txt", "w+")

        timeLimit = 120

        for i in range(0, 6):
            tmpFilePath = testGroupPath + "/" + testCaseFile[i]
            resultFile.write("==> File path: " + tmpFilePath + "\n")
            print("Solving:", tmpFilePath)
            solve(open(tmpFilePath, "r"), resultFile, timeLimit)

        resultFile.close()

if (__name__ == "__main__"):
    main()

Solving: /content/picked_test_cases/00Uncorrelated/n00050.kp
Solving: /content/picked_test_cases/00Uncorrelated/n02000.kp
Solving: /content/picked_test_cases/00Uncorrelated/n00100.kp
Solving: /content/picked_test_cases/00Uncorrelated/n05000.kp
Solving: /content/picked_test_cases/00Uncorrelated/n01000.kp
Solving: /content/picked_test_cases/00Uncorrelated/n00500.kp
Solving: /content/picked_test_cases/01WeaklyCorrelated/n00050.kp
Solving: /content/picked_test_cases/01WeaklyCorrelated/n02000.kp
Solving: /content/picked_test_cases/01WeaklyCorrelated/n00100.kp
Solving: /content/picked_test_cases/01WeaklyCorrelated/n05000.kp
Solving: /content/picked_test_cases/01WeaklyCorrelated/n01000.kp
Solving: /content/picked_test_cases/01WeaklyCorrelated/n00500.kp
Solving: /content/picked_test_cases/02StronglyCorrelated/n00050.kp
Solving: /content/picked_test_cases/02StronglyCorrelated/n02000.kp
Solving: /content/picked_test_cases/02StronglyCorrelated/n00100.kp
Solving: /content/picked_test_cases/02Stron

In [7]:
!zip -r /content/result.zip /content/_OR_Tools_Results

  adding: content/_OR_Tools_Results/ (stored 0%)
  adding: content/_OR_Tools_Results/12Circle.txt (deflated 71%)
  adding: content/_OR_Tools_Results/01WeaklyCorrelated.txt (deflated 71%)
  adding: content/_OR_Tools_Results/06UncorrelatedWithSimilarWeights.txt (deflated 72%)
  adding: content/_OR_Tools_Results/10MultipleStronglyCorrelated.txt (deflated 72%)
  adding: content/_OR_Tools_Results/08SpannerWeaklyCorrelated.txt (deflated 72%)
  adding: content/_OR_Tools_Results/07SpannerUncorrelated.txt (deflated 72%)
  adding: content/_OR_Tools_Results/04AlmostStronglyCorrelated.txt (deflated 71%)
  adding: content/_OR_Tools_Results/11ProfitCeiling.txt (deflated 71%)
  adding: content/_OR_Tools_Results/03InverseStronglyCorrelated.txt (deflated 72%)
  adding: content/_OR_Tools_Results/02StronglyCorrelated.txt (deflated 71%)
  adding: content/_OR_Tools_Results/09SpannerStronglyCorrelated.txt (deflated 73%)
  adding: content/_OR_Tools_Results/00Uncorrelated.txt (deflated 71%)
  adding: content/